# Model

Folder Structure:

#### Root
- Data/
- - Test/                      # Contains evaluation images
- - Train/                     # Contains training images
- - sample_submission.csv      # Sample submission file for Kaggle competition
- - train_labels.csv           # csv file containing training set labels in format (fileName, label)
- Model.ipynb                  # Current file


In [88]:
# Define constants
TRAIN_LABEL_FILE = 'Data\\train_labels.csv'
TRAIN_IMAGES_DIR = 'Data\\Train'
USE_GPU = False

In [89]:
# Imports
from __future__ import print_function, division

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, datasets

In [90]:
# Dataset Class
class PCamDataset(Dataset):
    """Histopathologic Cancer Detection Dataset"""
    def __init__(self, csv_file, root_dir, transform):
        
        """
        Args:
            csv_file (string): Path to the csv file with train labels.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.train_labels = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.train_labels)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.train_labels.iloc[idx, 0])
        img_name = img_name + '.tif'
        image = Image.open(img_name)
        label = self.train_labels.iloc[idx, 1:].values
        return self.transform(image), int(label)

In [91]:
# Model Class(es)
class CNN_V1(nn.Module):
    """Convolutional Neural Network"""
    def __init__(self):
        super(CNN_V1, self).__init__()
        # 1. Convolutional layers
        self.conv1 = nn.Conv2d( in_channels = 3
                               , out_channels = 16
                               , kernel_size = 3
                               , stride = 1
                               , padding = 1 )
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d( in_channels = 16
                               , out_channels = 32
                               , kernel_size = 3
                               , stride = 1
                               , padding = 1 )
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d( in_channels = 32
                               , out_channels = 64
                               , kernel_size = 3
                               , stride = 1
                               , padding = 1 )
        self.bn3 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d( kernel_size = 2
                                 , stride = 2
                                 , padding = 0 )

        # 2. FC layers to final output
        self.fc1 = nn.Linear(in_features = 64*12*12, out_features = 512)
        self.fc_bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(in_features = 512, out_features = 256)
        self.fc_bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(in_features = 256, out_features = 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Apply 3x...
        # Convolution Layers, followed by Batch Normalizations, Maxpool, and ReLU
        x = self.bn1(self.conv1(x))                      # batch_size x 96 x 96 x 16
        x = self.pool(F.relu(x))                         # batch_size x 48 x 48 x 16
        x = self.bn2(self.conv2(x))                      # batch_size x 48 x 48 x 32
        x = self.pool(F.relu(x))                         # batch_size x 24 x 24 x 32
        x = self.bn3(self.conv3(x))                      # batch_size x 24 x 24 x 64
        x = self.pool(F.relu(x))                         # batch_size x 12 x 12 x 64
        
        # Flatten the output for each image
        x = x.view(-1, self.num_flat_features(x))        # batch_size x 12*12*64
        
        # Apply 3 FC Layers
        x = self.fc1(x)
        x = self.fc_bn1(x)
        x = F.relu(x)
        x = self.fc_bn2(self.fc2(x))
        x = F.relu(x)
        x = self.fc3(x)
        return self.sigmoid(x)

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [92]:
# Helper functions
def show_image(image, label):
    """Show image with label"""
    print('Label: ' + str(label[0]))
    plt.imshow(image)
    
def test_accuracy(predicted, target):
    """Computes the accuracy"""
    predicted = (sigmoid(predicted.data.numpy()) > 0.5)
    true = target.data.numpy()
    accuracy = np.sum(predicted == true) / true.shape[0]
    true_positive_rate = np.sum((predicted == 1) * (true == 1)) / np.sum(true == 1)
    true_negative_rate = np.sum((predicted == 0) * (true == 0)) / np.sum(true == 0)
    return accuracy, true_positive_rate, true_negative_rate

def batch_accuracy(output, target):
    pred = torch.gt(output, 0.5)
    truth = torch.gt(target, 0.5)
    acc = pred.eq(truth).sum() / target.numel()
    return acc

def accuracy(data, target):
    """Computes the accuracy"""
    num_tensors = len(data)
    sum_acc = 0.0
    for tensor_idx in range(num_tensors):
        sum_acc += batch_accuracy(data[tensor_idx], target[tensor_idx])
    return sum_acc / num_tensors * 100

def train_dev_test_split_indices(dataset, train_split=0.8, dev_split=0.1, seed=30):
    """Given a dataset, returns the train/dev/test split indices"""
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    train_split_end = int(np.floor(train_split * dataset_size))
    dev_split_end = int(np.floor(dev_split * dataset_size)) + train_split_end
    np.random.seed(seed)
    np.random.shuffle(indices)
    train_indices = indices[:train_split_end]
    dev_indices = indices[train_split_end:dev_split_end]
    test_indices = indices[dev_split_end:]
    return train_indices, dev_indices, test_indices

In [93]:
print(CNN_V1())

CNN_V1(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=9216, out_features=512, bias=True)
  (fc_bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc_bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=256, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [94]:
# Transformations
x_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [95]:
# Model
model = CNN_V1()
if (USE_GPU):
    model.cuda()

In [96]:
# Optimizer and loss criterion
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr = lr)
criterion = nn.BCEWithLogitsLoss()

In [97]:
# Parameters
train_label_file = 'Data\\train_labels.csv'
train_set_dir = 'Data\\Train'

batch_size = 2
num_workers = 0
num_epochs = 10
early_stop_limit = 7
bad_epoch_count = 0
stop = False
train_loss_min = np.Inf

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [98]:
# Create DataSet
dataset = PCamDataset( csv_file = TRAIN_LABEL_FILE
                      , root_dir = TRAIN_IMAGES_DIR
                      , transform = transform )

# Create data indices for train/dev set split
train_indices, dev_indices, test_indices = train_dev_test_split_indices(dataset)

######################################################
### Starting with smaller samplers to ramp up ###
######################################################
train_subset_size = 10
dev_subset_size = 5
train_indices = train_indices[:train_subset_size]
dev_indices = dev_indices[:dev_subset_size]
######################################################

train_sampler = SubsetRandomSampler(train_indices) # 176,020 Images (Full) / [train_subset_size] Images (Subset)
dev_sampler   = SubsetRandomSampler(dev_indices)   # 22,002 Images (Full) / [dev_subset_size] Images (Subset)
test_sampler  = SubsetRandomSampler(test_indices)  # 22,003 Images

train_loader = DataLoader( dataset = dataset
                          , batch_size = batch_size
                          , num_workers = num_workers
                          , sampler = train_sampler )
dev_loader = DataLoader( dataset = dataset
                          , batch_size = batch_size
                          , num_workers = num_workers
                          , sampler = dev_sampler )

In [99]:
# Loop over the dataset multiple times
for epoch in range(num_epochs):
    
    # Keep track of training loss
    train_loss = 0.0
    Y_prediction_train = []
    Y_train = []
    
    # Keep track of dev loss
    dev_loss = 0.0
    Y_prediction_dev = []
    Y_dev = []

    # Train the model
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        if USE_GPU:
            data, target = image.cuda(), label.cuda()
        else:
            data, target = image, label
            
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        output = model(data)
        
        # Update target to be the same dimensions as output
        target = target.view(output.shape[0],1).float()
        
        # Store output and label to compute accuracy later
        Y_prediction_train.append(output)
        Y_train.append(target)

        # Calculate the batch's loss
        loss = criterion(output, target)
        
        # Backward pass
        loss.backward()
        
        # Perform a single optimization step to update parameters
        optimizer.step()
        
        # Update the training loss
        train_loss += loss.item()
        
    # Evaluate the model
    model.eval()
    with torch.no_grad():
        for batch_idx, (image, label) in enumerate(dev_loader):
            if USE_GPU:
                data, target = image.cuda(), label.cuda()
            else:
                data, target = image, label

            # Get predicted output
            output = model(data)

            # Update target to be the same dimensions as output
            target = target.view(output.shape[0],1).float()

            # Store output and label to compute accuracy later
            Y_prediction_dev.append(output)
            Y_dev.append(target)

            # Calculate the batch's loss
            loss = criterion(output, target)
            
            # Update the dev loss
            dev_loss += loss.item()
                  
    # Calculate average loss
    train_loss = train_loss/len(train_loader.dataset)
    dev_loss = dev_loss/len(dev_loader.dataset)
    
    # Output accuracy for epoch
    print('Train accuracy after {} epochs: {} %'
          .format(epoch, accuracy(Y_prediction_train, Y_train)))
    print('Dev accuracy after {} epochs: {} %'
          .format(epoch, accuracy(Y_prediction_dev, Y_dev)))
                  
    # Save model if train loss has decreased
    ### Focused on train loss at the moment because we want to overfit ###
    ### the training data before worrying about our variance.          ###
    if train_loss <= train_loss_min:
        print('Dev loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
              .format(train_loss_min, train_loss))
        torch.save(model.state_dict(), 'model.pt')
        train_loss_min = train_loss
        bad_epoch_count = 0
    # If train loss didn't improve, increase bad_epoch_count and stop if
    # bad_epoch_count >= early_stop_limit (early stop)
    else:
        bad_epoch_count += 1
        print('{} epochs of increasing dev loss'.format(bad_epoch_count))
        if (bad_epoch_count >= early_stop_limit):
            print('Stopping training')
            stop = True
        
    if (stop):
        break

Train accuracy after 0 epochs: 0 %
Dev accuracy after 0 epochs: 0 %
Dev loss decreased (inf --> 0.000020).  Saving model ...
Train accuracy after 1 epochs: 0 %
Dev accuracy after 1 epochs: 0 %
Dev loss decreased (0.000020 --> 0.000020).  Saving model ...
Train accuracy after 2 epochs: 0 %
Dev accuracy after 2 epochs: 0 %
Dev loss decreased (0.000020 --> 0.000019).  Saving model ...
Train accuracy after 3 epochs: 0 %
Dev accuracy after 3 epochs: 0 %
Dev loss decreased (0.000019 --> 0.000018).  Saving model ...
Train accuracy after 4 epochs: 0 %
Dev accuracy after 4 epochs: 0 %
Dev loss decreased (0.000018 --> 0.000018).  Saving model ...
Train accuracy after 5 epochs: 0 %
Dev accuracy after 5 epochs: 0 %
Dev loss decreased (0.000018 --> 0.000018).  Saving model ...
Train accuracy after 6 epochs: 0 %
Dev accuracy after 6 epochs: 0 %
Dev loss decreased (0.000018 --> 0.000017).  Saving model ...
Train accuracy after 7 epochs: 0 %
Dev accuracy after 7 epochs: 0 %
1 epochs of increasing dev 

In [ ]:
from torch.autograd import Variable
a = torch.empty(1, 1, dtype=torch.float)
a.fill_(0.5152)
print(a)

t = torch.empty(1, 1, dtype=torch.float)
t.fill_(0)
print(t)
print(nn.BCELoss()(a,t)) # Different numbers